In [1]:
import pandas as pd
import numpy as np


detailed_reg = pd.read_csv('/kaggle/input/march-machine-learning-mania-2024/MRegularSeasonDetailedResults.csv')
cities = pd.read_csv('/kaggle/input/march-machine-learning-mania-2024/Cities.csv')
conf_abbr = pd.read_csv('/kaggle/input/march-machine-learning-mania-2024/Conferences.csv')
conf_tourney = pd.read_csv('/kaggle/input/march-machine-learning-mania-2024/MConferenceTourneyGames.csv')
game_cities = pd.read_csv('/kaggle/input/march-machine-learning-mania-2024/MGameCities.csv')
massey = pd.read_csv('/kaggle/input/march-machine-learning-mania-2024/MMasseyOrdinals_thruSeason2024_day128.csv')
detailed_tourney = pd.read_csv('/kaggle/input/march-machine-learning-mania-2024/MNCAATourneyDetailedResults.csv')
team_confs = pd.read_csv('/kaggle/input/march-machine-learning-mania-2024/MTeamConferences.csv')


In [2]:
detailed_reg['Type'] = 'Season'
detailed_tourney['Type'] = 'Tourney'

combined_data = pd.concat([detailed_reg, detailed_tourney], ignore_index=True)

In [3]:
combined_data

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,Type
0,2003,10,1104,68,1328,62,N,0,27,58,...,16,22,10,22,8,18,9,2,20,Season
1,2003,10,1272,70,1393,63,N,0,26,62,...,9,20,20,25,7,12,8,6,16,Season
2,2003,11,1266,73,1437,61,N,0,24,58,...,14,23,31,22,9,12,2,5,23,Season
3,2003,11,1296,56,1457,50,N,0,18,38,...,8,15,17,20,9,19,4,3,23,Season
4,2003,11,1400,77,1208,71,N,0,30,61,...,17,27,21,15,12,10,7,1,14,Season
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114551,2023,146,1274,88,1400,81,N,0,29,49,...,11,15,9,14,20,13,6,1,23,Tourney
114552,2023,146,1361,57,1166,56,N,0,25,66,...,10,11,8,24,9,9,3,2,11,Tourney
114553,2023,152,1163,72,1274,59,N,0,28,57,...,12,12,12,17,10,9,8,1,12,Tourney
114554,2023,152,1361,72,1194,71,N,0,25,57,...,16,21,7,24,6,9,6,2,17,Tourney


In [4]:

win_columns = {
    'WTeamID': 'TeamID', 'WScore': 'Score', 'LTeamID': 'OpponentID', 'LScore': 'OpponentScore',
    'WFGM': 'FGM', 'WFGA': 'FGA', 'WFGM3': 'FGM3', 'WFGA3': 'FGA3', 'WFTM': 'FTM', 'WFTA': 'FTA',
    'WOR': 'OR', 'WDR': 'DR', 'WAst': 'Ast', 'WTO': 'TO', 'WStl': 'Stl', 'WBlk': 'Blk', 'WPF': 'PF',
    'LFGM': 'OpponentFGM', 'LFGA': 'OpponentFGA', 'LFGM3': 'OpponentFGM3', 'LFGA3': 'OpponentFGA3',
    'LFTM': 'OpponentFTM', 'LFTA': 'OpponentFTA', 'LOR': 'OpponentOR', 'LDR': 'OpponentDR', 
    'LAst': 'OpponentAst', 'LTO': 'OpponentTO', 'LStl': 'OpponentStl', 'LBlk': 'OpponentBlk', 'LPF': 'OpponentPF',
    'WLoc': 'Loc', 'NumOT': 'OT'
}

loss_columns = {
    'LTeamID': 'TeamID', 'LScore': 'Score', 'WTeamID': 'OpponentID', 'WScore': 'OpponentScore',
    'LFGM': 'FGM', 'LFGA': 'FGA', 'LFGM3': 'FGM3', 'LFGA3': 'FGA3', 'LFTM': 'FTM', 'LFTA': 'FTA',
    'LOR': 'OR', 'LDR': 'DR', 'LAst': 'Ast', 'LTO': 'TO', 'LStl': 'Stl', 'LBlk': 'Blk', 'LPF': 'PF',
    'WFGM': 'OpponentFGM', 'WFGA': 'OpponentFGA', 'WFGM3': 'OpponentFGM3', 'WFGA3': 'OpponentFGA3',
    'WFTM': 'OpponentFTM', 'WFTA': 'OpponentFTA', 'WOR': 'OpponentOR', 'WDR': 'OpponentDR', 
    'WAst': 'OpponentAst', 'WTO': 'OpponentTO', 'WStl': 'OpponentStl', 'WBlk': 'OpponentBlk', 'WPF': 'OpponentPF',
    'WLoc': 'Loc', 'NumOT': 'OT'
}

# Apply the renaming for wins and losses
wins_normalized = combined_data.rename(columns=win_columns)
losses_normalized = combined_data.rename(columns=loss_columns)

# Adjust the location for losses_normalized to reflect the perspective of the losing team
losses_normalized['Loc'] = losses_normalized['Loc'].replace({'H': 'A', 'A': 'H'})

# Concatenate the normalized dataframes
all_games_normalized = pd.concat([wins_normalized, losses_normalized], ignore_index=True)

# Sort by TeamID, Season, and DayNum to ensure chronological order for cumulative calculations
all_games_normalized.sort_values(by=['TeamID', 'Season', 'DayNum'], inplace=True)

# From here, you can proceed with calculating cumulative statistics, ensuring to group by 'TeamID' and 'Season'
# and then calculating the stats you're interested in (e.g., cumulative averages for scores, field goals, etc.)


In [5]:
all_games_normalized.columns

Index(['Season', 'DayNum', 'TeamID', 'Score', 'OpponentID', 'OpponentScore',
       'Loc', 'OT', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR',
       'Ast', 'TO', 'Stl', 'Blk', 'PF', 'OpponentFGM', 'OpponentFGA',
       'OpponentFGM3', 'OpponentFGA3', 'OpponentFTM', 'OpponentFTA',
       'OpponentOR', 'OpponentDR', 'OpponentAst', 'OpponentTO', 'OpponentStl',
       'OpponentBlk', 'OpponentPF', 'Type'],
      dtype='object')

In [6]:
stats_of_interest = ['Score', 'OpponentID', 'OpponentScore',
       'OT', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR',
       'Ast', 'TO', 'Stl', 'Blk', 'PF', 'OpponentFGM', 'OpponentFGA',
       'OpponentFGM3', 'OpponentFGA3', 'OpponentFTM', 'OpponentFTA',
       'OpponentOR', 'OpponentDR', 'OpponentAst', 'OpponentTO', 'OpponentStl',
       'OpponentBlk', 'OpponentPF']
for stat in stats_of_interest:
    all_games_normalized[f'CumulativeSum_{stat}'] = all_games_normalized.groupby(['TeamID', 'Season'])[stat].cumsum()

# Calculate the game count to use for average calculations
all_games_normalized['GameCount'] = all_games_normalized.groupby(['TeamID', 'Season']).cumcount() + 1

# Calculate cumulative averages for the statistics
for stat in stats_of_interest:
    all_games_normalized[f'CumulativeAvg_{stat}'] = all_games_normalized[f'CumulativeSum_{stat}'] / all_games_normalized['GameCount']


In [7]:
all_games_normalized

,Season,DayNum,TeamID,Score,OpponentID,OpponentScore,Loc,OT,FGM,FGA,...,CumulativeAvg_OpponentFGA3,CumulativeAvg_OpponentFTM,CumulativeAvg_OpponentFTA,CumulativeAvg_OpponentOR,CumulativeAvg_OpponentDR,CumulativeAvg_OpponentAst,CumulativeAvg_OpponentTO,CumulativeAvg_OpponentStl,CumulativeAvg_OpponentBlk,CumulativeAvg_OpponentPF
169822,2014,5,1101,75,1182,94,A,0,22,63,...,19.000000,27.000000,37.000000,10.000000,32.000000,19.000000,10.000000,4.000000,2.000000,22.000000
169892,2014,7,1101,47,1382,75,A,0,14,40,...,16.500000,25.000000,34.500000,14.000000,27.000000,17.000000,9.500000,5.500000,4.500000,22.500000
169982,2014,9,1101,44,1268,67,A,0,17,54,...,17.000000,20.333333,28.666667,14.000000,27.000000,15.333333,12.000000,6.666667,6.333333,17.666667
170149,2014,13,1101,41,1234,103,A,0,14,54,...,18.000000,19.750000,27.000000,11.750000,28.000000,17.750000,10.250000,7.750000,6.000000,15.750000
170249,2014,15,1101,64,1395,71,A,0,19,50,...,17.000000,20.600000,29.000000,11.000000,27.400000,16.400000,9.400000,7.600000,5.400000,16.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226820,2024,110,1478,58,1192,68,H,0,21,59,...,25.960000,11.480000,16.320000,9.400000,25.800000,15.200000,11.760000,6.360000,2.440000,13.960000
112554,2024,115,1478,70,1447,56,H,0,26,59,...,26.000000,11.538462,16.346154,9.423077,25.923077,14.846154,11.615385,6.346154,2.461538,13.884615
112716,2024,117,1478,74,1384,58,A,0,24,45,...,25.703704,11.444444,16.407407,9.444444,25.814815,14.666667,11.666667,6.407407,2.407407,13.962963
112835,2024,121,1478,82,1192,61,H,0,27,61,...,25.428571,11.607143,16.535714,9.535714,25.857143,14.535714,11.714286,6.392857,2.392857,14.071429


In [8]:
all_games_normalized['Win'] = (all_games_normalized['Score'] > all_games_normalized['OpponentScore']).astype(int)

# Assuming all_games_normalized has the 'Win' column updated based on scores

def rolling_wins(group, window_size):
    # Use the rolling function to sum 'Win' over the past 'window_size' games, filling NaNs with 0
    # The rolling window is not allowed to cross over into the next season's games
    wins = group['Win'].rolling(window=window_size, min_periods=1).sum().shift()
    return wins.fillna(0)

# Apply the rolling_wins function for each window size and create new columns in the DataFrame
window_sizes = [5, 10, 15]
for window_size in window_sizes:
    column_name = f'last{window_size}'
    all_games_normalized[column_name] = all_games_normalized.groupby(['TeamID', 'Season']).apply(rolling_wins, window_size).reset_index(level=[0,1], drop=True)

# Ensure the DataFrame is still sorted by TeamID, Season, and DayNum
all_games_normalized.sort_values(by=['TeamID', 'Season', 'DayNum'], inplace=True)

# After this, all_games_normalized will include the new features 'last5', 'last10', and 'last15' for each game


/tmp/ipykernel_25/4062309865.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  all_games_normalized[column_name] = all_games_normalized.groupby(['TeamID', 'Season']).apply(rolling_wins, window_size).reset_index(level=[0,1], drop=True)
/tmp/ipykernel_25/4062309865.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  all_games_normalized[column_name] = all_games_normalized.groupby(['TeamID', 'Season']).app

In [9]:
all_games_normalized

,Season,DayNum,TeamID,Score,OpponentID,OpponentScore,Loc,OT,FGM,FGA,...,CumulativeAvg_OpponentDR,CumulativeAvg_OpponentAst,CumulativeAvg_OpponentTO,CumulativeAvg_OpponentStl,CumulativeAvg_OpponentBlk,CumulativeAvg_OpponentPF,Win,last5,last10,last15
169822,2014,5,1101,75,1182,94,A,0,22,63,...,32.000000,19.000000,10.000000,4.000000,2.000000,22.000000,0,0.0,0.0,0.0
169892,2014,7,1101,47,1382,75,A,0,14,40,...,27.000000,17.000000,9.500000,5.500000,4.500000,22.500000,0,0.0,0.0,0.0
169982,2014,9,1101,44,1268,67,A,0,17,54,...,27.000000,15.333333,12.000000,6.666667,6.333333,17.666667,0,0.0,0.0,0.0
170149,2014,13,1101,41,1234,103,A,0,14,54,...,28.000000,17.750000,10.250000,7.750000,6.000000,15.750000,0,0.0,0.0,0.0
170249,2014,15,1101,64,1395,71,A,0,19,50,...,27.400000,16.400000,9.400000,7.600000,5.400000,16.600000,0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226820,2024,110,1478,58,1192,68,H,0,21,59,...,25.800000,15.200000,11.760000,6.360000,2.440000,13.960000,0,3.0,6.0,7.0
112554,2024,115,1478,70,1447,56,H,0,26,59,...,25.923077,14.846154,11.615385,6.346154,2.461538,13.884615,1,2.0,5.0,7.0
112716,2024,117,1478,74,1384,58,A,0,24,45,...,25.814815,14.666667,11.666667,6.407407,2.407407,13.962963,1,3.0,5.0,8.0
112835,2024,121,1478,82,1192,61,H,0,27,61,...,25.857143,14.535714,11.714286,6.392857,2.392857,14.071429,1,3.0,5.0,8.0


In [10]:
massey.columns

Index(['Season', 'RankingDayNum', 'SystemName', 'TeamID', 'OrdinalRank'], dtype='object')

In [11]:
massey['SystemName'].unique()

array(['SEL', 'AP', 'BIH', 'DUN', 'ENT', 'GRN', 'IMS', 'MAS', 'MKV',
       'MOR', 'POM', 'RPI', 'SAG', 'SAU', 'SE', 'STR', 'USA', 'WLK',
       'WOB', 'BOB', 'DWH', 'ERD', 'ECK', 'BRZ', 'ARG', 'RTH', 'WOL',
       'HOL', 'COL', 'DOL', 'GC', 'GRS', 'HER', 'TSR', 'MIC', 'WTE', 'BD',
       'MGY', 'RAG', 'CNG', 'SIM', 'DES', 'JON', 'LYN', 'NOR', 'RM',
       'REI', 'ACU', 'BCM', 'CMV', 'SAP', 'DC', 'KLK', 'WIL', 'ROH',
       'RIS', 'REN', 'SCR', 'DOK', 'PIG', 'KPK', 'PKL', 'TRX', 'MB',
       'JCI', 'PH', 'LYD', 'KRA', 'RTR', 'UCS', 'ISR', 'CPR', 'BKM',
       'JEN', 'REW', 'STH', 'SPW', 'RSE', 'PGH', 'CPA', 'RTB', 'TOL',
       'HKB', 'BPI', 'TW', 'NOL', 'DC2', 'DCI', 'OMY', 'EBB', 'LMC', 'RT',
       'KEL', 'KMV', 'RTP', 'TMR', 'AUS', 'ROG', 'BP5', 'PTS', 'KOS',
       'PEQ', 'ADE', 'BNM', 'CJB', 'BUR', 'HAT', 'MSX', 'BBT', '7OT',
       'SFX', 'EBP', 'TBD', 'CRO', 'D1A', 'TPR', 'BLS', 'DII', 'KBM',
       'TRP', 'LOG', 'SP', 'STF', 'WMR', 'PPR', 'MPI', 'STS', 'UPS',
       'SPR', 'Mv

In [12]:
# Assuming 'massey' is your DataFrame with rankings
# and 'all_games_normalized' is your main games DataFrame

# We'll first ensure that both DataFrames are sorted properly
massey.sort_values(by=['Season', 'TeamID', 'RankingDayNum'], inplace=True)
all_games_normalized.sort_values(by=['Season', 'TeamID', 'DayNum'], inplace=True)

# Merge asof to get the most recent ranking for each game
# 'asof_merge' requires sorted DataFrames
rankings_for_games = pd.merge_asof(
    all_games_normalized.sort_values('DayNum'),
    massey.sort_values('RankingDayNum'),
    by=['Season', 'TeamID'],
    left_on='DayNum',
    right_on='RankingDayNum',
    direction='backward'
)

# Now, 'rankings_for_games' contains the most recent ranking for each game
# But we need to pivot this so each SystemName has its own column
# Pivot the table to have SystemNames as columns and fill missing rankings with an appropriate value or NaN
pivot_rankings = rankings_for_games.pivot_table(
    index=['Season', 'TeamID', 'DayNum'],  # Preserving the game identifier
    columns='SystemName',  # Creating a column for each ranking system
    values='OrdinalRank'
).reset_index()

# Optionally, fill NaN values if needed, e.g., with forward fill within each season and team
pivot_rankings.fillna(method='ffill', inplace=True)
# Merge the pivoted rankings back onto the original games DataFrame
all_games_with_rankings = pd.merge(
    all_games_normalized,
    pivot_rankings,
    on=['Season', 'TeamID', 'DayNum'],
    how='left'
)

all_games_with_rankings = pd.merge(
    all_games_with_rankings,
    pivot_rankings.rename(columns={"TeamID":"OpponentID"}),
    on=['Season', 'OpponentID', 'DayNum'],
    how='left'
)

# 'all_games_with_rankings' now includes a column for each ranking system with the most recent ranking for each game


/tmp/ipykernel_25/805657548.py:29: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  pivot_rankings.fillna(method='ffill', inplace=True)


In [13]:
# Correctly use forward fill without 'method' parameter
all_games_with_rankings.sort_values(by=['Season', 'TeamID', 'DayNum'], inplace=True)
all_games_with_rankings.ffill(inplace=True)

# Fill remaining NaNs in ranking columns with the median value for each column, avoiding chained assignment
ranking_columns = [col for col in all_games_with_rankings.columns if '_x' in col or '_y' in col]

# Compute median values for ranking columns
median_ranks = all_games_with_rankings[ranking_columns].median()

# Use DataFrame.update() to fill NaNs with median values directly on the original DataFrame
all_games_with_rankings.update(all_games_with_rankings[ranking_columns].fillna(median_ranks))

In [14]:
all_games_with_rankings.dropna()

,Season,DayNum,TeamID,Score,OpponentID,OpponentScore,Loc,OT,FGM,FGA,...,WIL_y,WLK_y,WLS_y,WMR_y,WMV_y,WOB_y,WOL_y,WTE_y,YAG_y,ZAM_y
0,2003,19,1102,47,1257,65,A,0,16,40,...,190.0,165.0,74.0,197.0,158.0,156.0,182.0,177.0,328.0,286.0
1,2003,22,1102,72,1391,43,H,0,26,46,...,190.0,165.0,74.0,197.0,158.0,156.0,182.0,177.0,328.0,286.0
2,2003,25,1102,57,1117,52,A,0,16,36,...,190.0,165.0,74.0,197.0,158.0,156.0,182.0,177.0,328.0,286.0
3,2003,27,1102,47,1399,60,A,0,15,35,...,190.0,165.0,74.0,197.0,158.0,156.0,182.0,177.0,328.0,286.0
4,2003,31,1102,65,1410,44,H,0,23,40,...,190.0,165.0,74.0,197.0,158.0,156.0,182.0,177.0,328.0,286.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229107,2024,110,1478,58,1192,68,H,0,21,59,...,318.0,136.0,74.0,197.0,347.0,287.0,239.0,177.0,328.0,286.0
229108,2024,115,1478,70,1447,56,H,0,26,59,...,285.0,212.0,74.0,197.0,347.0,287.0,219.0,177.0,328.0,286.0
229109,2024,117,1478,74,1384,58,A,0,24,45,...,178.0,86.0,74.0,197.0,347.0,287.0,8.0,177.0,328.0,286.0
229110,2024,121,1478,82,1192,61,H,0,27,61,...,318.0,136.0,74.0,197.0,347.0,287.0,239.0,177.0,328.0,286.0


In [15]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import numpy as np
import pandas as pd

def preprocess_data(all_games_with_rankings, test_size=0.2, random_state=42):
    """
    Preprocesses the game data for neural network training.
    
    Parameters:
    - all_games_with_rankings: DataFrame containing game statistics and rankings.
    - test_size: Proportion of the dataset to include in the test split.
    - random_state: Controls the shuffling applied to the data before applying the split.
    
    Returns:
    - X_train_scaled: Scaled training features.
    - X_val_scaled: Scaled validation features.
    - y_train: Training target.
    - y_val: Validation target.
    - encoder: Fitted OneHotEncoder instance.
    - scaler: Fitted StandardScaler instance.
    """
    features = all_games_with_rankings.drop(['Win'], axis=1)  # Drop the target column
    target = all_games_with_rankings['Win'].values  # This is your target variable

    # One-hot encode the 'Type' column
    encoder = OneHotEncoder(sparse=False)
    type_encoded = encoder.fit_transform(features[['Type']])

    # Get one-hot encoded feature names
    type_feature_labels = encoder.get_feature_names_out(['Type'])

    # Concatenate the one-hot encoded 'Type' back with the rest of your numeric features
    numeric_features = features.drop(['Score','OpponentScore','Loc',
                                      'OT','FGM','FGA','FGM3','FGA3','FTM','FTA',
                                      'OR','DR','Ast','TO','Stl','Blk','PF','OpponentFGM',
                                      'OpponentFGA','OpponentFGM3','OpponentFGA3',
                                      'OpponentFTM','OpponentFTA','OpponentOR','OpponentDR',
                                      'OpponentAst','OpponentTO','OpponentStl','OpponentBlk',
                                      'OpponentPF','Type'], axis=1).select_dtypes(include=[np.number])
    
    numeric_features2 = numeric_features.copy()
    merged = pd.merge(numeric_features, numeric_features2, left_on=['Season', 'DayNum', 'TeamID'], right_on=['Season', 'DayNum', 'OpponentID']).drop(columns= ["Season", "DayNum", 'TeamID_x', 'OpponentID_x', 'TeamID_y', 'OpponentID_y'])
    print(len(list(merged.columns)))

    numeric_features_array = merged.values
    features_encoded = np.concatenate([numeric_features_array, type_encoded], axis=1)
    

    # Split the data into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(features_encoded, target, test_size=test_size, random_state=random_state)

    # Scale the numeric features (excluding the one-hot encoded 'Type' columns)
    scaler = StandardScaler()
    N = features_encoded.shape[1]  # Number of numeric features
    X_train_scaled = np.copy(X_train)
    X_val_scaled = np.copy(X_val)
    X_train_scaled[:, :N] = scaler.fit_transform(X_train[:, :N])
    X_val_scaled[:, :N] = scaler.transform(X_val[:, :N])

    return X_train_scaled, X_val_scaled, y_train, y_val, encoder, scaler


In [16]:
X_train_scaled, X_val_scaled, y_train, y_val, encoder, scaler = preprocess_data(all_games_with_rankings)

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


884


In [17]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from kerastuner.tuners import RandomSearch

def build_model(hp):
    model = Sequential()
    model.add(Dense(units=hp.Int('units_input', min_value=32, max_value=256, step=32),
                    activation='relu', input_shape=(X_train_scaled.shape[1],)))
    
    for i in range(hp.Int('n_layers', 1, 3)):
        model.add(Dense(units=hp.Int(f'units_{i}', min_value=32, max_value=256, step=32),
                        activation='relu'))
        model.add(Dropout(rate=hp.Float(f'dropout_{i}', min_value=0.0, max_value=0.5, default=0.5, step=0.1)))
    
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer=Adam(hp.Float('learning_rate', 1e-4, 1e-2, sampling='log')),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model


2024-03-18 23:03:33.827626: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-18 23:03:33.827722: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-18 23:03:33.990521: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/tmp/ipykernel_25/1926199156.py:6: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [18]:
import kerastuner as kt

tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=1,  # Set more trials for a more thorough search.
    executions_per_trial=1,
    directory='my_dir',
    project_name='hparam_tuning3'
)

# Early stopping callback to prevent overfitting
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

# Start hyperparameter search
tuner.search(X_train_scaled, y_train,
             epochs=5,
             validation_data=(X_val_scaled, y_val),
             callbacks=[stop_early],
             verbose=1)

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Best hyperparameters:\n{best_hps.values}")

# Build the model with the best hyperparameters and train it on the data
best_model = tuner.hypermodel.build(best_hps)
history = best_model.fit(X_train_scaled, y_train,
                         validation_data=(X_val_scaled, y_val),
                         epochs=5,  # You may adjust this based on the early stopping callback
                         verbose=1)

# Optionally, evaluate the best model
val_loss, val_accuracy = best_model.evaluate(X_val_scaled, y_val, verbose=0)
print(f'Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}')


Trial 1 Complete [00h 01m 01s]
val_accuracy: 0.7653798460960388

Best val_accuracy So Far: 0.7653798460960388
Total elapsed time: 00h 01m 01s
Best hyperparameters:
{'units_input': 64, 'n_layers': 1, 'units_0': 192, 'dropout_0': 0.0, 'learning_rate': 0.0010309260084789597}
Epoch 1/5
  84/5728 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - accuracy: 0.5748 - loss: 0.6692

W0000 00:00:1710803089.746650      73 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


5728/5728 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7312 - loss: 0.5165

W0000 00:00:1710803098.888969      71 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1710803099.626805      73 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


5728/5728 ━━━━━━━━━━━━━━━━━━━━ 13s 2ms/step - accuracy: 0.7312 - loss: 0.5165 - val_accuracy: 0.7563 - val_loss: 0.4842
Epoch 2/5
 101/5728 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - accuracy: 0.7654 - loss: 0.4584

W0000 00:00:1710803101.514709      71 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


5728/5728 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - accuracy: 0.7634 - loss: 0.4726 - val_accuracy: 0.7620 - val_loss: 0.4773
Epoch 3/5
5728/5728 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - accuracy: 0.7709 - loss: 0.4616 - val_accuracy: 0.7646 - val_loss: 0.4734
Epoch 4/5
5728/5728 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - accuracy: 0.7763 - loss: 0.4527 - val_accuracy: 0.7651 - val_loss: 0.4718
Epoch 5/5
5728/5728 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - accuracy: 0.7820 - loss: 0.4425 - val_accuracy: 0.7643 - val_loss: 0.4749
Validation Loss: 0.47485828399658203, Validation Accuracy: 0.7643322944641113


In [19]:
teams = pd.read_csv('/kaggle/input/march-machine-learning-mania-2024/MTeams.csv')

In [20]:
numeric_feature_names = list(all_games_with_rankings.drop(columns = ['Score','OpponentScore','Loc',
                                      'OT','FGM','FGA','FGM3','FGA3','FTM','FTA',
                                      'OR','DR','Ast','TO','Stl','Blk','PF','OpponentFGM',
                                      'OpponentFGA','OpponentFGM3','OpponentFGA3',
                                      'OpponentFTM','OpponentFTA','OpponentOR','OpponentDR',
                                      'OpponentAst','OpponentTO','OpponentStl','OpponentBlk',
                                      'OpponentPF','Type', 'Win']).columns)

In [21]:
def matchup_predictor(team_a_id, team_b_id, season, day_num, model, encoder, scaler):
    combined_features = generate_game_features(team_a_id, team_b_id, season, day_num, encoder, scaler, numeric_feature_names)
    # Ensure combined_features is correctly shaped for model input
    combined_features_reshaped = scaler.transform(combined_features)
    #print(combined_features)
    win_probability = model.predict(combined_features_reshaped)[0]
    return win_probability

In [22]:
def generate_game_features(team_id_a, team_id_b, season, day_num, encoder, scaler, numeric_feature_names):
    
    # Existing code to filter all_games_with_rankings for a specific season, team, and before a certain day
    team_a_data = all_games_with_rankings[
        (all_games_with_rankings['Season'] == season) &
        (all_games_with_rankings['TeamID'] == team_a_id) &
        (all_games_with_rankings['DayNum'] < day_num)
    ]

    # New code to filter team_a_data to only include the row with the max DayNum
    if not team_a_data.empty:
        max_day_num_index = team_a_data['DayNum'].idxmax()
        team_a_data = team_a_data.loc[[max_day_num_index]]

    team_b_data = all_games_with_rankings[
        (all_games_with_rankings['Season'] == season) &
        ((all_games_with_rankings['TeamID'] == team_b_id)) &
        (all_games_with_rankings['DayNum'] < day_num)
    ]
    
    if not team_b_data.empty:
        max_day_num_index = team_b_data['DayNum'].idxmax()
        team_b_data = team_b_data.loc[[max_day_num_index]]

    if team_a_data.empty:
        raise ValueError(f"No data found for team ID {team_a_id} in season {season}.")
    if team_b_data.empty:
        raise ValueError(f"No data found for team ID {team_b_id} in season {season}.")

    # Assuming 'Type' is a column in team_data, create a DataFrame for encoding
    # Ensure this matches how 'Type' was handled during model training
    
    merged = pd.merge(team_a_data[numeric_feature_names], team_b_data[numeric_feature_names], on = "Season").drop(columns= ['Season', 'DayNum_x', 'TeamID_x', 'OpponentID_x', 'DayNum_y', 'TeamID_y', 'OpponentID_y'])

    type_df = team_a_data[['Type']].drop_duplicates()  # Adjust as necessary

    type_encoded = encoder.transform(type_df)

    # Calculate mean of numeric features
    numeric_means = merged[merged.columns].mean().to_frame().T
    #print(team_a_data[numeric_feature_names])

    # Concatenate encoded 'Type' with numeric features BEFORE scaling
    combined_features = np.concatenate([numeric_means, type_encoded], axis=1)

    # Scale the combined features
    #features_scaled = scaler.transform(combined_features)
    
    #print(team_id_a, team_id_b)
    return combined_features

In [23]:

season = 2024
day_num = 999
team_a_id = 1104
team_b_id = 1120

# Predicting the win probability for Team A against Team B
win_probability = matchup_predictor(team_a_id, team_b_id, season, day_num, best_model, encoder, scaler)

print(f"The probability of Team A (ID: {team_a_id}) winning against Team B (ID: {team_b_id}) is {win_probability[0]:.2}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step
The probability of Team A (ID: 1104) winning against Team B (ID: 1120) is 0.22


In [24]:
def predict_game_winner(team_a_id, team_b_id, season, daynum, model, encoder, scaler):
    """
    Wrapper for the matchup_predictor to return the winning team based on the predicted probability.
    Incorporates the day number into the prediction.
    """
    # Assuming the matchup_predictor function returns a probability of team_a winning
    print(team_a_id, team_b_id)
    win_probability = matchup_predictor(team_a_id, team_b_id, season, daynum, model, encoder, scaler)
    # Randomly determine the game outcome based on win probability
    return team_a_id if np.random.rand() < win_probability else team_b_id

In [25]:
structure = pd.read_csv('/kaggle/input/march-machine-learning-mania-2024/MNCAATourneySeedRoundSlots.csv').drop(columns = ["EarlyDayNum", "LateDayNum"])
seeds_csv = pd.read_csv('/kaggle/input/march-machine-learning-mania-2024/2024_tourney_seeds.csv')
seeds_csv = seeds_csv[seeds_csv["Tournament"] == 'M']

In [26]:
import pandas as pd
import numpy as np
from itertools import combinations

def generate_round_slots():
    slots = []
    # Define the number of teams and initialize matchups for the first round
    num_teams = 64
    regions = ['W', 'X', 'Y', 'Z']
    region_teams = num_teams // len(regions)  # Teams per region
    
    # Generate first-round matchups for each region
    for region in regions:
        for i in range(1, region_teams // 2 + 1):
            strong_seed = f"{region}{i:02}"
            weak_seed = f"{region}{region_teams + 1 - i:02}"
            slot = f"R1{region}{i}"
            slots.append({"Slot": slot, "StrongSeed": strong_seed, "WeakSeed": weak_seed})
    
    # Dynamically generate matchups for subsequent rounds until the championship
    rounds = int(np.log2(num_teams))  # Total rounds needed for the championship
    for round_num in range(2, rounds + 1):
        round_slots_per_region = 2 ** (rounds - round_num)  # Slots decrease by half each round
        for region in regions:
            for i in range(1, round_slots_per_region + 1):
                # For simplicity in identifying slots, use region and round number
                slot = f"R{round_num}{region}{i}"
                # Seed placeholders now simply reference the slot they advanced from
                strong_seed = f"R{round_num-1}{region}{2*i-1}"
                weak_seed = f"R{round_num-1}{region}{2*i}"
                slots.append({"Slot": slot, "StrongSeed": strong_seed, "WeakSeed": weak_seed})
    
    # Final Four and Championship (assuming regions W & X and Y & Z face off)
    final_four = [
        ("R6WX1", "R5W1", "R5X1"),
        ("R6YZ1", "R5Y1", "R5Z1"),
    ]
    for slot, strong_seed, weak_seed in final_four:
        slots.append({"Slot": slot, "StrongSeed": strong_seed, "WeakSeed": weak_seed})
    
    # Championship
    slots.append({"Slot": "R7CH1", "StrongSeed": "R6WX1", "WeakSeed": "R6YZ1"})
    
    return pd.DataFrame(slots)


# Example: Generate round slots for a 64-team tournament
round_slots = generate_round_slots()
round_slots[:40]

,Slot,StrongSeed,WeakSeed
0,R1W1,W01,W16
1,R1W2,W02,W15
2,R1W3,W03,W14
3,R1W4,W04,W13
4,R1W5,W05,W12
5,R1W6,W06,W11
6,R1W7,W07,W10
7,R1W8,W08,W09
8,R1X1,X01,X16
9,R1X2,X02,X15


In [27]:
structure[structure["GameSlot"] == "R1W1"]

,Seed,GameRound,GameSlot
0,W01,1,R1W1
174,W16,1,R1W1
181,W16a,1,R1W1
188,W16b,1,R1W1


In [28]:
import pandas as pd

def run_tournament_simulation(seeds, round_slots, model, encoder, scaler, season, daynum, tournament_type, bracket_num):
    seeds_dict = seeds.set_index('Seed')['TeamID'].to_dict()
    # Initialize a dictionary to map slots to team IDs dynamically
    slot_to_teamID = {seed: teamID for seed, teamID in seeds_dict.items()}
    
    winners = []
    for _, row in round_slots.iterrows():
        # Use the updated dynamic mapping for team IDs
        strong_team_id = slot_to_teamID.get(row['StrongSeed'])
        weak_team_id = slot_to_teamID.get(row['WeakSeed'])
        
        if not strong_team_id or not weak_team_id:
            print("E: " + row['StrongSeed'] + " " + row['WeakSeed'])
            continue
        
        winner_id = predict_game_winner(strong_team_id, weak_team_id, season, daynum, model, encoder, scaler)
        
        # Determine the winner's seed and update the dynamic mapping for the next round
        winner_seed = next((seed for seed, id in seeds_dict.items() if id == winner_id), None)
        slot_to_teamID[row['Slot']] = winner_id  # Update for next round's matchups
        
        winners.append((row['Slot'], winner_seed))
    
    # Convert the winners list to a DataFrame for submission
    winners_df = pd.DataFrame(winners, columns=['Slot', 'Team'])
    winners_df['Bracket'] = bracket_num
    winners_df['Tournament'] = tournament_type
    winners_df.reset_index(inplace=True, drop=True)
    winners_df.index.name = 'RowId'

    return winners_df

def simulate_n_brackets(n, seeds, round_slots, model, encoder, scaler, season, daynum, tournament_type='M'):
    all_brackets = pd.DataFrame()
    
    for bracket_num in range(1, n + 1):
        bracket_results = run_tournament_simulation(seeds, round_slots, model, encoder, scaler, season, daynum, tournament_type, bracket_num)
        bracket_results['Bracket'] = bracket_num  # Tag each row with its bracket number
        
        all_brackets = pd.concat([all_brackets, bracket_results], ignore_index=True)
    
    all_brackets.reset_index(inplace=True)
    all_brackets.rename(columns={'index': 'RowId'}, inplace=True)
    
    return all_brackets


In [29]:
round_slots

,Slot,StrongSeed,WeakSeed
0,R1W1,W01,W16
1,R1W2,W02,W15
2,R1W3,W03,W14
3,R1W4,W04,W13
4,R1W5,W05,W12
...,...,...,...
154,R6Y1,R5Y1,R5Y2
155,R6Z1,R5Z1,R5Z2
156,R6WX1,R5W1,R5X1
157,R6YZ1,R5Y1,R5Z1


In [30]:
simulate_n_brackets(5, seeds_csv, round_slots, best_model, encoder, scaler, 2024, 999, 'M')

1163 1391
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1235 1355
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1228 1287
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1120 1463
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1361 1412
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1140 1182
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1450 1179
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1194 1321
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1314 1224
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1112 1253
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1124 1159
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1104 1158
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1388 1213
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1155 1307
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1173 1305
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1280 1277
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1345 1212
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1397 1389
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1166 1103
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1242 1359
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1211 1270
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16

,RowId,Slot,Team,Bracket,Tournament
0,0,R1W1,W16,1,M
1,1,R1W2,W15,1,M
2,2,R1W3,W14,1,M
3,3,R1W4,W13,1,M
4,4,R1W5,W05,1,M
...,...,...,...,...,...
295,295,R3Z2,Z08,5,M
296,296,R4W1,W09,5,M
297,297,R4X1,X13,5,M
298,298,R4Y1,Y11,5,M


In [31]:
teams[teams["TeamID"] == 1222]

,TeamID,TeamName,FirstD1Season,LastD1Season
121,1222,Houston,1985,2024


In [32]:
round_slots = pd.read_csv('/kaggle/input/march-machine-learning-mania-2024/MNCAATourneySlots.csv')
round_slots = round_slots[round_slots["Season"] == 2024].drop(columns = ["Season"]).reset_index().drop(columns = ["index"])

In [33]:
round_slots[:30]

,Slot,StrongSeed,WeakSeed
0,R1W1,W01,W16
1,R1W2,W02,W15
2,R1W3,W03,W14
3,R1W4,W04,W13
4,R1W5,W05,W12
5,R1W6,W06,W11
6,R1W7,W07,W10
7,R1W8,W08,W09
8,R1X1,X01,X16
9,R1X2,X02,X15
